In [2]:
import time
import json
import requests
import urllib.request
import os
from bs4 import BeautifulSoup
from bs4 import Comment
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import pickle

In [3]:
pd.set_option('display.max_colwidth', None)


In [7]:
pip install webdriver-manager


  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.


# Sofa

In [9]:
import os
import time
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

# Set up ChromeDriver properly
chrome_driver_path = "/usr/local/bin/chromedriver"
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

# Define directory
data_dir = 'sofa'
product_dir = os.path.join('images', data_dir)
os.makedirs(product_dir, exist_ok=True)

# URL to scrape
url = 'https://www.allmodern.com/furniture/cat/sofas-sectionals-c527685.html#sbprodgrid'
driver.get(url)

# Create dataframe
full_df_sofa = pd.DataFrame(columns=['id', 'website_link', 'image_link', 'prices', 'category'])
counter = 0  

for i in range(1, 11):
    time.sleep(2)

    # Scroll down to load images
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    img_container = soup.find_all('div', class_='Category-productWrap')

    image_links, prices, website_links, ids = [], [], [], []

    for img_div in img_container:
        # Get image link
        image_tag = img_div.find('img', src=True)
        if image_tag:
            image_url = image_tag['src']
            if 'data:image' not in image_url:
                image_filename = f"{product_dir}/{data_dir}_{counter}.jpg"
                urllib.request.urlretrieve(image_url, image_filename)
                image_links.append(image_url)
                ids.append(f"{data_dir}_{counter}")
                counter += 1

        # Get product link
        link_tag = img_div.find('a', href=True)
        if link_tag:
            website_links.append(link_tag['href'])

        # Get price
        price_tag = img_div.find('span', class_='ProductCard-price')
        if price_tag:
            prices.append(price_tag.text.strip())

    # Store in DataFrame
    data = pd.DataFrame({
        'id': ids,
        'website_link': website_links,
        'image_link': image_links,
        'prices': prices,
        'category': data_dir
    })

    full_df_sofa = pd.concat([full_df_sofa, data], ignore_index=True)

    # Pagination
    try:
        next_button = driver.find_element(By.XPATH, '//*[@id="bd"]/div[2]/div[2]/nav/a[{}]'.format(i))
        ActionChains(driver).move_to_element(next_button).click().perform()
    except:
        print("No more pages or navigation failed.")
        break

    time.sleep(5)

driver.quit()


NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [5]:
data_dir = 'sofa'

product_dir = os.path.join('images', data_dir)
counter = 0
if not os.path.exists(product_dir):
    os.makedirs(product_dir)

path = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(path)

url = 'https://www.allmodern.com/furniture/cat/sofas-sectionals-c527685.html#sbprodgrid'
driver.get(url)

full_df_sofa = pd.DataFrame(columns=['id', 'website_link', 'image_link', 'prices', 'category'])
counter = 0 

for i in range(1, 11):
    time.sleep(2)
    #driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    if  5 > i >= 2:
        i = i+1
        
    elif 9 > i >= 5:
        i = 4
    
    elif i == 9:
        i = 5

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    img = soup.find('div', {'class': 'Category-productWrap is-loaded'})

    image_link = []
    prices = []
    website_link = []
    id = []

    for image in img.findAll('img', attrs={'srcset':True}):
        image = image.attrs['srcset'][1+image.attrs['srcset'].find(','):image.attrs['srcset'].find('.310w')-4]
        if ('data:image') in image:
                pass
        else:
            urllib.request.urlretrieve(image, product_dir+'/'+data_dir+str(counter)+'.jpg')
            image_link.append(image)
            id.append(data_dir+str(counter))
            counter += 1
    
    for website in img.findAll('a', {'href':True}):
        website = website['href']
        website_link.append(website)


    for price in img.findAll('div',{'class':'ProductCard-pricing'}):
        price = price.find('span', {'class':'ProductCard-price'}).text
        prices.append(price)

    data = pd.DataFrame(dict(zip(['id', 'website_link', 'image_link', 'prices'],[id, website_link, image_link, prices])))
    data['category'] = data_dir
    
    full_df_sofa = full_df_sofa.append(data)

    if i == 10:
        break
    
    element = driver.find_element_by_xpath('//*[@id="bd"]/div[2]/div[2]/nav/a[{}]'.format(i))
    element.click()

    time.sleep(10)

driver.quit()

AttributeError: 'str' object has no attribute 'capabilities'

# Chairs

In [16]:
full_df_chair = pd.DataFrame(columns=['website_link', 'image_link', 'prices', 'category'])

data_dir = 'chairs'

product_dir = os.path.join('images', data_dir)
counter = 0
if not os.path.exists(product_dir):
    os.makedirs(product_dir)

path = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(path)

url = 'https://www.allmodern.com/furniture/sb0/accent-chairs-c365818.html'
driver.get(url)

counter = 0 

for i in range(1, 11):
    time.sleep(2)
    #driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    if  5 > i >= 2:
        i = i+1
        
    elif 9 > i >= 5:
        i = 4
    
    elif i == 9:
        i = 5

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    img = soup.find('div', {'data-hb-id': 'pl-grid-v2'})

    image_link = []
    prices = []
    website_link = []

    for image in img.findAll('img', attrs={'srcset':True}):
        image = image.attrs['srcset'][1+image.attrs['srcset'].find(','):image.attrs['srcset'].find('.310w')-4]
        if ('data:image') in image:
                pass
        else:
            urllib.request.urlretrieve(image, product_dir+'/'+data_dir+str(counter)+'.jpg')
            image_link.append(image)
            counter += 1
    
    for website in img.findAll('a', {'href':True}):
        website = website['href']
        website_link.append(website)

    if i == 1:
        for price in img.findAll('div',{'class':'ProductCard-pricing'}):
            price = price.find('span', {'class':'ProductCard-price'}).text
            prices.append(price)    
    else:
        for price in img.findAll('div',{'class':'BrowseMinimizedPriceBlock'}):
            price = price.find('span', {'class':'BrowseMinimizedPriceBlock-price'}).text
            prices.append(price)


    data = pd.DataFrame(dict(zip(['website_link', 'image_link', 'prices'],[website_link, image_link, prices])))
    data['category'] = data_dir
    full_df_chair = full_df_chair.append(data)

    if i == 10:
        break

    element = driver.find_element_by_xpath('//*[@id="sbprodgrid"]/div[3]/div/nav/a[{}]'.format(i))
    element.click()

    time.sleep(10)

driver.quit()

# Console Tables

In [ ]:
full_df_consoletable = pd.DataFrame(columns=['website_link', 'image_link', 'prices', 'category'])

data_dir = 'console table'

product_dir = os.path.join('images', data_dir)
counter = 0
if not os.path.exists(product_dir):
    os.makedirs(product_dir)

path = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(path)

url = 'https://www.allmodern.com/furniture/sb0/console-tables-c413349.html'
driver.get(url)

counter = 0 

for i in range(1, 5):
    time.sleep(2)
    #driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    if  5 > i >= 2:
        i = i+1

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    img = soup.find('div', {'data-hb-id': 'pl-grid-v2'})

    image_link = []
    prices = []
    website_link = []

    for image in img.findAll('img', attrs={'srcset':True}):
        image = image.attrs['srcset'][1+image.attrs['srcset'].find(','):image.attrs['srcset'].find('.310w')-4]
        if ('data:image') in image:
                pass
        else:
            urllib.request.urlretrieve(image, product_dir+'/'+data_dir+str(counter)+'.jpg')
            image_link.append(image)
            counter += 1
    
    for website in img.findAll('a', {'href':True}):
        website = website['href']
        website_link.append(website)

    if i == 1:
        for price in img.findAll('div',{'class':'ProductCard-pricing'}):
            price = price.find('span', {'class':'ProductCard-price'}).text
            prices.append(price)    
    else:
        for price in img.findAll('div',{'class':'BrowseMinimizedPriceBlock'}):
            price = price.find('span', {'class':'BrowseMinimizedPriceBlock-price'}).text
            prices.append(price)


    data = pd.DataFrame(dict(zip(['website_link', 'image_link', 'prices'],[website_link, image_link, prices])))
    data['category'] = data_dir
    full_df_consoletable = full_df_consoletable.append(data)
    
    if i == 5:
        break
    element = driver.find_element_by_xpath('//*[@id="sbprodgrid"]/div[3]/div/nav/a[{}]'.format(i))
    element.click()

    time.sleep(10)

driver.quit()

# TV Stands

In [3]:
full_df_tvstand = pd.DataFrame(columns=['website_link', 'image_link', 'prices', 'category'])

data_dir = 'tv stand'

product_dir = os.path.join('images', data_dir)
counter = 0
if not os.path.exists(product_dir):
    os.makedirs(product_dir)

path = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(path)

url = 'https://www.allmodern.com/furniture/sb0/tv-stands-c445006.html'
driver.get(url)

counter = 0 

for i in range(1, 5):
    time.sleep(2)
    #driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    if  5 > i >= 2:
        i = i+1

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    img = soup.find('div', {'data-hb-id': 'pl-grid-v2'})

    image_link = []
    prices = []
    website_link = []

    for image in img.findAll('img', attrs={'srcset':True}):
        image = image.attrs['srcset'][1+image.attrs['srcset'].find(','):image.attrs['srcset'].find('.310w')-4]
        if ('data:image') in image:
                pass
        else:
            urllib.request.urlretrieve(image, product_dir+'/'+data_dir+str(counter)+'.jpg')
            image_link.append(image)
            counter += 1
    
    for website in img.findAll('a', {'href':True}):
        website = website['href']
        website_link.append(website)

    if i == 1:
        for price in img.findAll('div',{'class':'ProductCard-pricing'}):
            price = price.find('span', {'class':'ProductCard-price'}).text
            prices.append(price)    
    else:
        for price in img.findAll('div',{'class':'BrowseMinimizedPriceBlock'}):
            price = price.find('span', {'class':'BrowseMinimizedPriceBlock-price'}).text
            prices.append(price)


    data = pd.DataFrame(dict(zip(['website_link', 'image_link', 'prices'],[website_link, image_link, prices])))
    data['category'] = data_dir
    full_df_tvstand = full_df_tvstand.append(data)
    
    if i == 5:
        break
    element = driver.find_element_by_xpath('//*[@id="sbprodgrid"]/div[3]/div/nav/a[{}]'.format(i))
    element.click()

    time.sleep(10)

driver.quit()

# Coffee Tables

In [25]:
full_df_coffeetable = pd.DataFrame(columns=['website_link', 'image_link', 'prices', 'category'])

data_dir = 'coffee tables'

product_dir = os.path.join('images', data_dir)
counter = 0
if not os.path.exists(product_dir):
    os.makedirs(product_dir)

path = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(path)

url = 'https://www.allmodern.com/furniture/sb0/coffee-tables-c413347.html'
driver.get(url)

counter = 0 

for i in range(1, 10):
    time.sleep(2)
    #driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    if  5 > i >= 2:
        i = i+1
        
    elif 8 > i >= 5:
        i = 4
    
    elif i == 8:
        i = 5

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    img = soup.find('div', {'data-hb-id': 'pl-grid-v2'})

    image_link = []
    prices = []
    website_link = []

    for image in img.findAll('img', attrs={'srcset':True}):
        image = image.attrs['srcset'][1+image.attrs['srcset'].find(','):image.attrs['srcset'].find('.310w')-4]
        if ('data:image') in image:
                pass
        else:
            urllib.request.urlretrieve(image, product_dir+'/'+data_dir+str(counter)+'.jpg')
            image_link.append(image)
            counter += 1
    
    for website in img.findAll('a', {'href':True}):
        website = website['href']
        website_link.append(website)

    if i == 1:
        for price in img.findAll('div',{'class':'ProductCard-pricing'}):
            price = price.find('span', {'class':'ProductCard-price'}).text
            prices.append(price)    
    else:
        for price in img.findAll('div',{'class':'BrowseMinimizedPriceBlock'}):
            price = price.find('span', {'class':'BrowseMinimizedPriceBlock-price'}).text
            prices.append(price)


    data = pd.DataFrame(dict(zip(['website_link', 'image_link', 'prices'],[website_link, image_link, prices])))
    data['category'] = data_dir
    full_df_coffeetable = full_df_coffeetable.append(data)

    if i == 9:
        break
    element = driver.find_element_by_xpath('//*[@id="sbprodgrid"]/div[3]/div/nav/a[{}]'.format(i))
    element.click()
    
    time.sleep(10)

driver.quit()

# End Tables

In [28]:
full_df_endtable = pd.DataFrame(columns=['website_link', 'image_link', 'prices', 'category'])

data_dir = 'end tables'

product_dir = os.path.join('images', data_dir)
counter = 0
if not os.path.exists(product_dir):
    os.makedirs(product_dir)

path = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(path)

url = 'https://www.allmodern.com/furniture/sb0/end-side-tables-c413348.html'
driver.get(url)

counter = 0 

for i in range(1, 10):
    time.sleep(2)
    #driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    if  5 > i >= 2:
        i = i+1
        
    elif 8 > i >= 5:
        i = 4

    elif i == 8:
        i = 5

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    img = soup.find('div', {'data-hb-id': 'pl-grid-v2'})

    image_link = []
    prices = []
    website_link = []

    for image in img.findAll('img', attrs={'srcset':True}):
        image = image.attrs['srcset'][1+image.attrs['srcset'].find(','):image.attrs['srcset'].find('.310w')-4]
        if ('data:image') in image:
                pass
        else:
            urllib.request.urlretrieve(image, product_dir+'/'+data_dir+str(counter)+'.jpg')
            image_link.append(image)
            counter += 1
    
    for website in img.findAll('a', {'href':True}):
        website = website['href']
        website_link.append(website)

    if i == 1:
        for price in img.findAll('div',{'class':'ProductCard-pricing'}):
            price = price.find('span', {'class':'ProductCard-price'}).text
            prices.append(price)    
    else:
        for price in img.findAll('div',{'class':'BrowseMinimizedPriceBlock'}):
            price = price.find('span', {'class':'BrowseMinimizedPriceBlock-price'}).text
            prices.append(price)


    data = pd.DataFrame(dict(zip(['website_link', 'image_link', 'prices'],[website_link, image_link, prices])))
    data['category'] = data_dir
    full_df_endtable = full_df_endtable.append(data)

    if i == 9:
        break

    element = driver.find_element_by_xpath('//*[@id="sbprodgrid"]/div[3]/div/nav/a[{}]'.format(i))
    element.click()
    
    time.sleep(10)

driver.quit()

# Ottomans

In [30]:
full_df_ottomans = pd.DataFrame(columns=['website_link', 'image_link', 'prices', 'category'])

data_dir = 'ottomans'

product_dir = os.path.join('images', data_dir)
counter = 0
if not os.path.exists(product_dir):
    os.makedirs(product_dir)

options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled') 
path = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(path, options=options)

url = 'https://www.allmodern.com/furniture/sb0/ottomans-poufs-c365824.html'
driver.get(url)

counter = 0 

for i in range(1, 6):
    time.sleep(2)
    #driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    if  6 > i >= 2:
        i = i+1

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    img = soup.find('div', {'data-hb-id': 'pl-grid-v2'})

    image_link = []
    prices = []
    website_link = []

    for image in img.findAll('img', attrs={'srcset':True}):
        image = image.attrs['srcset'][1+image.attrs['srcset'].find(','):image.attrs['srcset'].find('.310w')-4]
        if ('data:image') in image:
                pass
        else:
            urllib.request.urlretrieve(image, product_dir+'/'+data_dir+str(counter)+'.jpg')
            image_link.append(image)
            counter += 1
    
    for website in img.findAll('a', {'href':True}):
        website = website['href']
        website_link.append(website)

    if i == 1:
        for price in img.findAll('div',{'class':'ProductCard-pricing'}):
            price = price.find('span', {'class':'ProductCard-price'}).text
            prices.append(price)    
    else:
        for price in img.findAll('div',{'class':'BrowseMinimizedPriceBlock'}):
            price = price.find('span', {'class':'BrowseMinimizedPriceBlock-price'}).text
            prices.append(price)


    data = pd.DataFrame(dict(zip(['website_link', 'image_link', 'prices'],[website_link, image_link, prices])))
    data['category'] = data_dir
    full_df_ottomans = full_df_ottomans.append(data)
    
    if i == 6:
        break
    
    element = driver.find_element_by_xpath('//*[@id="sbprodgrid"]/div[3]/div/nav/a[{}]'.format(i))
    element.click()

    time.sleep(10)

driver.quit()

# Lamps

In [33]:
full_df_lamp = pd.DataFrame(columns=['website_link', 'image_link', 'prices', 'category'])

data_dir = 'lamp'

product_dir = os.path.join('images', data_dir)
counter = 0
if not os.path.exists(product_dir):
    os.makedirs(product_dir)

path = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(path)

url = 'https://www.allmodern.com/lighting/sb0/table-lamps-c1872230.html'
driver.get(url)

counter = 0 

for i in range(1, 5):
    time.sleep(2)
    #driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    if  5 > i >= 2:
        i = i+1

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    img = soup.find('div', {'data-hb-id': 'pl-grid-v2'})

    image_link = []
    prices = []
    website_link = []

    for image in img.findAll('img', attrs={'srcset':True}):
        image = image.attrs['srcset'][1+image.attrs['srcset'].find(','):image.attrs['srcset'].find('.310w')-4]
        if ('data:image') in image:
                pass
        else:
            urllib.request.urlretrieve(image, product_dir+'/'+data_dir+str(counter)+'.jpg')
            image_link.append(image)
            counter += 1
    
    for website in img.findAll('a', {'href':True}):
        website = website['href']
        website_link.append(website)

    if i == 1:
        for price in img.findAll('div',{'class':'ProductCard-pricing'}):
            price = price.find('span', {'class':'ProductCard-price'}).text
            prices.append(price)    
    else:
        for price in img.findAll('div',{'class':'BrowseMinimizedPriceBlock'}):
            price = price.find('span', {'class':'BrowseMinimizedPriceBlock-price'}).text
            prices.append(price)


    data = pd.DataFrame(dict(zip(['website_link', 'image_link', 'prices'],[website_link, image_link, prices])))
    data['category'] = data_dir
    full_df_lamp = full_df_lamp.append(data)
    
    if i == 5:
        break
    element = driver.find_element_by_xpath('//*[@id="sbprodgrid"]/div[3]/div/nav/a[{}]'.format(i))
    element.click()

    time.sleep(10)

driver.quit()

In [36]:
#full_df_lamp = pd.DataFrame(columns=['website_link', 'image_link', 'prices', 'category'])

data_dir = 'lamp'

# product_dir = os.path.join('images', data_dir)
# counter = 0
# if not os.path.exists(product_dir):
#     os.makedirs(product_dir)

path = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(path)

url = 'https://www.allmodern.com/lighting/sb0/floor-lamps-c477075.html'
driver.get(url)

for i in range(1, 4):
    time.sleep(2)
    #driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    if  4 > i >= 2:
        i = i+1

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    img = soup.find('div', {'data-hb-id': 'pl-grid-v2'})

    image_link = []
    prices = []
    website_link = []

    for image in img.findAll('img', attrs={'srcset':True}):
        image = image.attrs['srcset'][1+image.attrs['srcset'].find(','):image.attrs['srcset'].find('.310w')-4]
        if ('data:image') in image:
                pass
        else:
            urllib.request.urlretrieve(image, product_dir+'/'+data_dir+str(counter)+'.jpg')
            image_link.append(image)
            counter += 1
    
    for website in img.findAll('a', {'href':True}):
        website = website['href']
        website_link.append(website)

    if i == 1:
        for price in img.findAll('div',{'class':'ProductCard-pricing'}):
            price = price.find('span', {'class':'ProductCard-price'}).text
            prices.append(price)    
    else:
        for price in img.findAll('div',{'class':'BrowseMinimizedPriceBlock'}):
            price = price.find('span', {'class':'BrowseMinimizedPriceBlock-price'}).text
            prices.append(price)


    data = pd.DataFrame(dict(zip(['website_link', 'image_link', 'prices'],[website_link, image_link, prices])))
    data['category'] = data_dir
    full_df_lamp = full_df_lamp.append(data)
    
    if i == 4:
        break
    element = driver.find_element_by_xpath('//*[@id="sbprodgrid"]/div[3]/div/nav/a[{}]'.format(i))
    element.click()

    time.sleep(10)

driver.quit()

# Dining Tables

In [40]:
full_df_diningtable = pd.DataFrame(columns=['website_link', 'image_link', 'prices', 'category'])

data_dir = 'dining tables'

product_dir = os.path.join('images', data_dir)
counter = 0
if not os.path.exists(product_dir):
    os.makedirs(product_dir)

path = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(path)

url = 'https://www.allmodern.com/furniture/sb0/dining-tables-c366120.html'
driver.get(url)

counter = 0 

for i in range(1, 9):
    time.sleep(2)
    #driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    if  5 > i >= 2:
        i = i+1
        
    elif 7 > i >= 5:
        i = 4
    
    elif i == 7:
        i = 5

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    img = soup.find('div', {'data-hb-id': 'pl-grid-v2'})

    image_link = []
    prices = []
    website_link = []

    for image in img.findAll('img', attrs={'srcset':True}):
        image = image.attrs['srcset'][1+image.attrs['srcset'].find(','):image.attrs['srcset'].find('.310w')-4]
        if ('data:image') in image:
                pass
        else:
            urllib.request.urlretrieve(image, product_dir+'/'+data_dir+str(counter)+'.jpg')
            image_link.append(image)
            counter += 1
    
    for website in img.findAll('a', {'href':True}):
        website = website['href']
        website_link.append(website)

    if i == 1:
        for price in img.findAll('div',{'class':'ProductCard-pricing'}):
            price = price.find('span', {'class':'ProductCard-price'}).text
            prices.append(price)    
    else:
        for price in img.findAll('div',{'class':'BrowseMinimizedPriceBlock'}):
            price = price.find('span', {'class':'BrowseMinimizedPriceBlock-price'}).text
            prices.append(price)


    data = pd.DataFrame(dict(zip(['website_link', 'image_link', 'prices'],[website_link, image_link, prices])))
    data['category'] = data_dir
    full_df_diningtable = full_df_diningtable.append(data)
    
    if i == 8:
        break
    element = driver.find_element_by_xpath('//*[@id="sbprodgrid"]/div[3]/div/nav/a[{}]'.format(i))
    element.click()

    time.sleep(10)

driver.quit()

In [41]:
len(full_df_diningtable)

347

# Dining chair

In [42]:
full_df_diningchair = pd.DataFrame(columns=['website_link', 'image_link', 'prices', 'category'])

data_dir = 'dining chair'

product_dir = os.path.join('images', data_dir)
counter = 0
if not os.path.exists(product_dir):
    os.makedirs(product_dir)

path = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(path)

url = 'https://www.allmodern.com/furniture/sb0/dining-chairs-c366121.html'
driver.get(url)

counter = 0 

for i in range(1, 7):
    time.sleep(2)
    #driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    if  i >= 2:
        i = i+1

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    img = soup.find('div', {'data-hb-id': 'pl-grid-v2'})

    image_link = []
    prices = []
    website_link = []

    for image in img.findAll('img', attrs={'srcset':True}):
        image = image.attrs['srcset'][1+image.attrs['srcset'].find(','):image.attrs['srcset'].find('.310w')-4]
        if ('data:image') in image:
                pass
        else:
            urllib.request.urlretrieve(image, product_dir+'/'+data_dir+str(counter)+'.jpg')
            image_link.append(image)
            counter += 1
    
    for website in img.findAll('a', {'href':True}):
        website = website['href']
        website_link.append(website)


    if i == 1:
        for price in img.findAll('div',{'class':'ProductCard-pricing'}):
            price = price.find('span', {'class':'ProductCard-price'}).text
            prices.append(price)    
    else:
        for price in img.findAll('div',{'class':'BrowseMinimizedPriceBlock'}):
            price = price.find('span', {'class':'BrowseMinimizedPriceBlock-price'}).text
            prices.append(price)


    data = pd.DataFrame(dict(zip(['website_link', 'image_link', 'prices'],[website_link, image_link, prices])))
    data['category'] = data_dir
    full_df_diningchair = full_df_diningchair.append(data)
    
    if i == 7:
        break
    element = driver.find_element_by_xpath('//*[@id="sbprodgrid"]/div[3]/div/nav/a[{}]'.format(i))
    element.click()

    time.sleep(10)

driver.quit()

In [44]:
a = full_df_diningchair.append(full_df_diningtable)

In [59]:
a['prices'].apply(lambda x: x.split()[1] if len(x.split()) > 1 else x.split()[0])

0        $98
1       $130
2        $73
3       $320
4       $181
       ...  
6     $1,820
7       $900
8     $1,449
9       $980
10    $1,000
Name: prices, Length: 612, dtype: object

# Rug

In [61]:
full_df_rug = pd.DataFrame(columns=['website_link', 'image_link', 'prices', 'category'])

data_dir = 'rug'

product_dir = os.path.join('images', data_dir)
counter = 0
if not os.path.exists(product_dir):
    os.makedirs(product_dir)

path = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(path)

url = 'https://www.allmodern.com/rugs/sb0/area-rugs-c409269.html'
driver.get(url)

counter = 0 

for i in range(1, 13):
    time.sleep(2)
    #driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    if  5 > i >= 2:
        i = i+1
        
    elif 11 > i >= 5:
        i = 4

    elif i == 11:
        i = 5

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    img = soup.find('div', {'data-hb-id': 'pl-grid-v2'})

    image_link = []
    prices = []
    website_link = []

    for image in img.findAll('img', attrs={'srcset':True}):
        image = image.attrs['srcset'][1+image.attrs['srcset'].find(','):image.attrs['srcset'].find('.310w')-4]
        if ('data:image') in image:
                pass
        else:
            urllib.request.urlretrieve(image, product_dir+'/'+data_dir+str(counter)+'.jpg')
            image_link.append(image)
            counter += 1
    
    for website in img.findAll('a', {'href':True}):
        website = website['href']
        website_link.append(website)

    if i == 1:
        for price in img.findAll('div',{'class':'ProductCard-pricing'}):
            price = price.find('span', {'class':'ProductCard-price'}).text
            prices.append(price)    
    else:
        for price in img.findAll('div',{'class':'BrowseMinimizedPriceBlock'}):
            price = price.find('span', {'class':'BrowseMinimizedPriceBlock-price'}).text
            prices.append(price)


    data = pd.DataFrame(dict(zip(['website_link', 'image_link', 'prices'],[website_link, image_link, prices])))
    data['category'] = data_dir
    full_df_rug = full_df_rug.append(data)

    if i == 12:
        break

    element = driver.find_element_by_xpath('//*[@id="sbprodgrid"]/div[3]/div/nav/a[{}]'.format(i))
    element.click()
    
    time.sleep(10)

driver.quit()

# Wall Art


In [ ]:
full_df_wallart = pd.DataFrame(columns=['website_link', 'image_link', 'prices', 'category'])

data_dir = 'wall art'

product_dir = os.path.join('images', data_dir)
counter = 0
if not os.path.exists(product_dir):
    os.makedirs(product_dir)

path = "/usr/local/bin/chromedriver"
driver = webdriver.Chrome(path)

url = 'https://www.allmodern.com/decor-pillows/sb0/wall-art-c446716.html?prefetch=true'
driver.get(url)

counter = 0 

for i in range(1, 16):
    time.sleep(2)
    #driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    if  5 > i >= 2:
        i = i+1
        
    elif 14 > i >= 5:
        i = 4

    elif i == 14:
        i = 5

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    img = soup.find('div', {'data-hb-id': 'pl-grid-v2'})

    image_link = []
    prices = []
    website_link = []

    for image in img.findAll('img', attrs={'srcset':True}):
        image = image.attrs['srcset'][1+image.attrs['srcset'].find(','):image.attrs['srcset'].find('.310w')-4]
        if ('data:image') in image:
                pass
        else:
            urllib.request.urlretrieve(image, product_dir+'/'+data_dir+str(counter)+'.jpg')
            image_link.append(image)
            counter += 1
    
    for website in img.findAll('a', {'href':True}):
        website = website['href']
        website_link.append(website)

    # if i == 1:
    for price in img.findAll('div',{'class':'ProductCard-pricing'}):
        price = price.find('span', {'class':'ProductCard-price'}).text
        prices.append(price)    
    # else:
    #     for price in img.findAll('div',{'class':'BrowseMinimizedPriceBlock'}):
    #         price = price.find('span', {'class':'BrowseMinimizedPriceBlock-price'}).text
    #         prices.append(price)


    data = pd.DataFrame(dict(zip(['website_link', 'image_link', 'prices'],[website_link, image_link, prices])))
    data['category'] = data_dir
    full_df_wallart = full_df_wallart.append(data)

    if i == 15:
        break

    element = driver.find_element_by_xpath('//*[@id="sbprodgrid"]/div[3]/div/nav/a[{}]'.format(i))
    element.click()
    
    time.sleep(10)

driver.quit()

In [76]:
list = [full_df_rug, full_df_lamp, full_df_diningchair, full_df_diningtable, full_df_sofa, full_df_chair, full_df_coffeetable, full_df_consoletable, full_df_endtable, full_df_ottomans, full_df_tvstand]

full_df = pd.DataFrame(columns=['website_link', 'image_link', 'prices', 'category'])

for i in list:
    full_df = full_df.append(i)

In [ ]:
df = pickle.load(open('full_df.p', 'rb'))

grouped = df.groupby('category', as_index=False)
grouped_index = grouped.apply(lambda x: x.reset_index(drop = True)).reset_index()
result = grouped_index.drop('level_0',axis = 1).set_index('level_1').reset_index()
result['id'] = result.apply(lambda x: x['category']+str(x['level_1']), axis=1)

pickle.dump(result, open('result.p', 'wb'))